# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [40]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [41]:
#QUESTÃO 1
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'

r = requests.get(url)
r

data_json = r.json()
data_json['items']

tabela_df = pd.DataFrame(data_json['items'])

In [42]:
#QUESTÃO 1

tabala_rj_municipio = tabela_df
tabala_rj_municipio = tabala_rj_municipio[tabala_rj_municipio['uf'] == 'RJ']

tabela_rj_municipio_interessado = tabala_rj_municipio.copy()

tabela_rj_municipio_interessado['fl_tipo_interessado'] = 0
tabela_rj_municipio_interessado.loc[tabela_rj_municipio_interessado['tipo_interessado'] == 'Município', 'fl_tipo_interessado'] = 1 
tabela_rj_resultado = tabela_rj_municipio_interessado[tabela_rj_municipio_interessado['fl_tipo_interessado'] == 1]

tabela_rj_resultado[['tipo_interessado', 'uf', 'num_pvl']].head(2)

,tipo_interessado,uf,num_pvl
81,Município,RJ,None
277,Município,RJ,PVL02.007214/2022-19


In [43]:
#QUESTÃO 2
tabela_rj = tabela_df

tabela_rj['fl_estado'] = 0
tabela_rj.loc[tabela_rj['uf'] == 'RJ', 'fl_estado'] = 1
tabela_filtrada_rj = tabela_rj[tabela_rj['fl_estado'] == 1]

tabela_filtrada_rj.head(2)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status,fl_estado
81,5290,Município,Casimiro de Abreu,3301306,RJ,None,Deferido,17944.000078/2014-81,2014-11-28T02:00:00Z,Operação contratual interna,PAC 2 - Pró-Transporte - Pavimentação e Qualif...,Instituição Financeira Nacional,Caixa Econômica Federal,Real,15024627.28,1,0,12/12/2014,1
277,61009,Município,Rio das Flores,3304508,RJ,PVL02.007214/2022-19,Arquivado por decurso de prazo,17944.103071/2022-20,2022-08-12T18:48:50Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,6000000.00,0,0,10/02/2023,1


In [44]:
tabela_filtrada_rj['status'].value_counts()

status
Deferido                                                 24
Arquivado                                                11
Regularizado                                             10
Encaminhado à PGFN com manifestação técnica favorável     7
Arquivado por decurso de prazo                            6
Deferido (PVL-IF)                                         5
Arquivado a pedido                                        1
Name: count, dtype: int64

In [45]:
#QUESTÃO 2
porcentagem = tabela_filtrada_rj['status'].value_counts(normalize=True) * 100
porcentagem_formatada = porcentagem.map("{:.2f}%".format)
print(f'A Porcentagem de cada é:\n{porcentagem_formatada.to_string()}')

A Porcentagem de cada é:
status
Deferido                                                 37.50%
Arquivado                                                17.19%
Regularizado                                             15.62%
Encaminhado à PGFN com manifestação técnica favorável    10.94%
Arquivado por decurso de prazo                            9.38%
Deferido (PVL-IF)                                         7.81%
Arquivado a pedido                                        1.56%


In [46]:
#QUESTÃO 3
tabela_filtrada_att = tabela_filtrada_rj[:].copy()
tabela_filtrada_att['data_status'] = tabela_filtrada_att['data_status'].str.slice(6, 11)
tabela_filtrada_att['data_status'].value_counts()

data_status
2008    9
2011    7
2012    7
2013    6
2007    6
2023    5
2009    4
2010    4
2019    3
2020    3
2016    2
2014    2
2015    2
2002    1
2022    1
2018    1
2006    1
Name: count, dtype: int64

In [47]:
#QUESTÃO 4
porcentagem_ano = tabela_filtrada_att['data_status'].value_counts(normalize=True)*100
porcentagem_ano_formatado = porcentagem_ano.map('{:.2f}%'.format)
print (f'A porcentagem de cada ano é: {porcentagem_ano_formatado.to_string()}')

A porcentagem de cada ano é: data_status
2008    14.06%
2011    10.94%
2012    10.94%
2013     9.38%
2007     9.38%
2023     7.81%
2009     6.25%
2010     6.25%
2019     4.69%
2020     4.69%
2016     3.12%
2014     3.12%
2015     3.12%
2002     1.56%
2022     1.56%
2018     1.56%
2006     1.56%


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [48]:
#QUESTÃO 1
df_questao2 = pd.DataFrame(data_json['items'])

In [49]:
tabela_escolha = df_questao2

In [50]:
estado = 'BA'

tabela_escolha['fl_estado'] = 0
tabela_escolha.loc[tabela_escolha['uf'] == estado, 'fl_estado'] = 1
tabela_escolha_estado = tabela_escolha[tabela_escolha['fl_estado'] == 1]


tabela_escolha_estado.head(2)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status,fl_estado
16,60731,Município,Barra da Estiva,2902807,BA,PVL02.007122/2022-21,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,4000000.0,0,1,15/07/2022,1
26,64401,Município,Barreiras,2903201,BA,PVL02.001147/2023-00,Deferido,17944.101999/2023-51,2023-05-10T13:55:39Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,40000000.0,0,1,17/05/2023,1


In [51]:
copia_tabela_escolha_estado = tabela_escolha_estado.copy()

interessado = 'Estado'

copia_tabela_escolha_estado['fl_interessado'] = 0
copia_tabela_escolha_estado.loc[copia_tabela_escolha_estado['tipo_interessado'] == interessado, 'fl_interessado'] = 1
tabela_escolha_interessado = copia_tabela_escolha_estado[copia_tabela_escolha_estado['fl_interessado'] == 1]

tabela_escolha_interessado.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status,fl_estado,fl_interessado
843,8949,Estado,Bahia,29,BA,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000872/2014-24,2014-07-11T03:00:00Z,Operação contratual externa (com garantia da U...,Saúde,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,2.000000e+08,1,0,28/07/2014,1,1
977,7433,Estado,Bahia,29,BA,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000575/2012-17,2012-08-20T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,7.000000e+08,1,0,06/09/2012,1,1
1033,8342,Estado,Bahia,29,BA,None,Deferido,17944.000760/2009-14,2009-12-07T02:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Dólar dos EUA,6.289455e+06,1,0,17/12/2009,1,1
1418,9412,Estado,Bahia,29,BA,None,Deferido,17944.000975/2010-61,2010-06-30T03:00:00Z,Operação contratual interna,PAC - Copa Pró-Transporte,Instituição Financeira Nacional,Caixa Econômica Federal,Real,5.418000e+08,1,0,21/07/2010,1,1
1643,10420,Estado,Bahia,29,BA,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001195/2012-08,2013-07-04T03:00:00Z,Operação contratual externa (com garantia da U...,Profisco,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,4.527000e+07,1,0,17/07/2013,1,1


In [52]:
#QUESTÃO 2
tabela_mg = pd.DataFrame(data_json['items'])

In [53]:
#Filtro por estado, criar coluna fl_estado com valor 0 e adicionar valor 1 somente ao estado de MG
tabela_mg['fl_estado'] = 0
tabela_mg.loc[tabela_mg['uf'] == 'MG', 'fl_estado'] = 1
tabela_mg_filtrada = tabela_mg[tabela_mg['fl_estado'] == 1]

#copia da tabela acima agora acrecentando uma coluna chamada fl_tipo_status com valor 0 
tabela_mg_filtrada_tipo = tabela_mg_filtrada.copy()

#Filtro por status, criar coluna fl_tipo_status com valor 0 e adicionar valor 1 somente ao status desejado
tabela_mg_filtrada_tipo['fl_tipo_status'] = 0
tabela_mg_filtrada_tipo.loc[tabela_mg_filtrada_tipo['status'] == 'Arquivado por decurso de prazo', 'fl_tipo_status'] = 1
tabela_mg_filtrada_status = tabela_mg_filtrada_tipo[tabela_mg_filtrada_tipo['fl_tipo_status'] == 1]

#S 
tabela_mg_filtrada_status['fl_tipo_status'].sum()

73

In [54]:
#QUESTÃO 2.2 Outro método
tabela_mg_filtrada['status'].value_counts()

status
Deferido                                                 466
Deferido (PVL-IF)                                        268
Arquivado                                                 97
Arquivado por decurso de prazo                            73
Regularizado                                              24
Encaminhado à PGFN com manifestação técnica favorável     14
Arquivado a pedido                                        10
Arquivado por decurso de prazo (PVL-IF)                    7
Em retificação pelo interessado (PVL-IF)                   6
Indeferido                                                 3
Cancelado                                                  2
Em retificação pelo credor (PVL-IF)                        2
Em retificação pelo interessado                            1
Em retificação pelo credor                                 1
Name: count, dtype: int64

In [55]:
#QUESTÃO 2.3

tabala_bahia = tabela_df
tabela_bahia_filtro = tabela_df[tabela_df['uf'] == 'BA']
tabela_bahia_municipio = tabela_bahia_filtro.copy()
tabela_bahia_municipio['fl_status'] = 0
tabela_bahia_municipio.loc[tabela_bahia_municipio['status'] == 'Deferido', 'fl_status'] = 1 
tabela_bahia_cidade = tabela_bahia_municipio[tabela_bahia_municipio['fl_status'] == 1]
tabela_bahia_cidade['interessado'].value_counts()

interessado
Bahia                   12
Lauro de Freitas         3
Riacho de Santana        3
Vitória da Conquista     3
Campo Formoso            3
                        ..
Formosa do Rio Preto     1
Barra do Choça           1
Elísio Medrado           1
Malhada                  1
Guajeru                  1
Name: count, Length: 83, dtype: int64

In [57]:
#QUESTÃO 2.4
tabala_bahia = tabela_df
tabela_bahia_filtro = tabela_df[tabela_df['uf'] == 'BA']
tabela_bahia_municipio = tabela_bahia_filtro.copy()
tabela_bahia_municipio['fl_tipo_interessado'] = 0
tabela_bahia_municipio.loc[tabela_bahia_municipio['tipo_interessado'] == 'Estado', 'fl_tipo_interessado'] = 1 
tabela_bahia_interessado = tabela_bahia_municipio[tabela_bahia_municipio['fl_tipo_interessado'] == 1]
tabela_bahia_interessado.to_csv('tabela_bahia_interessado.csv')